# Map

> In this example, we will scrape Deliveroo for additional information on dark kitchen locations to map all sites in the UK.

In [ ]:
#| default_exp fsa

In [ ]:
import csv
import requests 
from bs4 import BeautifulSoup
from nbdev.showdoc import *
from deliveroo_editions.selenium_utils import *
from deliveroo_editions.deliveroo_utils import *
import pandas as pd
import datetime
from tqdm import tqdm
import urllib
import re
import json

Lets read in a csv file containing all the Deliveroo Editions kitchens scraped on the 10th July 2023:

In [ ]:
csv_file = 'data/editions_2023_07_10_11hr.csv'
data = []
columns_to_read = ['name', 'location', 'edition', 'restaurant_url']

with open(csv_file, 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        selected_data = {col: row[col] for col in columns_to_read}
        data.append(selected_data)

In [ ]:
# extract the address
for i, edition in enumerate(tqdm(data)):
    data[i]['address'] = get_address_from_restaurant_url(data[i]['restaurant_url'])

100%|█████████████████████████████████████████| 197/197 [16:49<00:00,  5.13s/it]


In [ ]:
# Save the list of dictionaries to a file
filename = 'data/editions_addressed_2023_07_10_11hr.csv'  # Specify the filename or path
with open(filename, 'w') as file:
    json.dump(data, file)

In [ ]:
### wip from here: 

In [ ]:
def get_coords_from_address(address_string):
    """Attempt retrieval of coords for input address string."""
    address = urllib.parse.quote(address_string)
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
    response = requests.get(url).json()
    if response:
        return {'lat': response[0]['lat'], 'lon': response[0]['lon'], 'address': address_string}
    else:
        postcode_string = infer_postcode(address_string)
        if postcode_string:
            url = "http://api.postcodes.io/postcodes/" + urllib.parse.quote(postcode_string)
            response = requests.get(url).json()
            if str(response['status']) == '200':
                return {'lat': response['result']['latitude'], 'lon': response['result']['longitude'], 'postcode': postcode_string}
            else:
                # try nearby postcode:
                nearby_postcode = get_nearby_postcode(postcode_string)
                if nearby_postcode:
                    url = "http://api.postcodes.io/postcodes/" + urllib.parse.quote(nearby_postcode)
                    response = requests.get(url).json()
                    if str(response['status']) == "200":
                        return {'lat': response['result']['latitude'], 'lon': response['result']['longitude'], 'postcode': nearby_postcode}
                else:
                    print("failed")
        else:
            # print("No postcode found for: " + address_string)
            pass

In [ ]:
def infer_postcode(address_string):
    """Extracts UK postcode from input address string with regex."""
    postcode = re.findall(r'\b[A-Z]{1,2}[0-9][A-Z0-9]? [0-9][ABD-HJLNP-UW-Z]{2}\b', address_string)
    if postcode:
        return postcode[0]
    else:
        return